Importing reqd modules

In [1]:
import numpy as np
import heapq
import random
import nltk
from nltk.corpus import words

### Loading/generating transition matrix and vocabulary data

#### For loading data

Enter input: L, n+2, transition matrix, vocabulary

In [2]:
L = int(input("L (vocabulary size):"))
n = int(input("n+2 (sentence length): ")) - 2
tmatrix_path = input("transition.txt path: ")
vocab_path = input("vocabulary.txt path: ")

L (vocabulary size): 4
n+2 (sentence length):  6
transition.txt path:  sample_transition.txt
vocabulary.txt path:  sample_vocab.txt


In [3]:
def load_data(vocab_file, transition_file):
    with open(vocab_file, 'r') as vf:
        vocab = vf.read().splitlines()
    
    transition_matrix = np.loadtxt(transition_file)
    return vocab, transition_matrix

In [4]:
vocab, tmatrix = load_data(vocab_path, tmatrix_path)
print(vocab)
print(tmatrix)

# bringing row that has P(wi|w0) values at the top to ease indexing
row0 = tmatrix[len(tmatrix) - 2]
tmatrix = np.delete(tmatrix, len(tmatrix) - 2, axis = 0)
tmatrix = np.insert(tmatrix, 0, row0, axis = 0)
print()
print(tmatrix)

['green', 'grass', 'at', 'airport']
[[0.6324 0.1576 0.4218 0.8491]
 [0.0975 0.9706 0.9157 0.934 ]
 [0.2785 0.9572 0.7922 0.6787]
 [0.5469 0.4854 0.9595 0.7577]
 [0.9575 0.8003 0.6557 0.7431]
 [0.9649 0.1419 0.0357 0.3922]]

[[0.9575 0.8003 0.6557 0.7431]
 [0.6324 0.1576 0.4218 0.8491]
 [0.0975 0.9706 0.9157 0.934 ]
 [0.2785 0.9572 0.7922 0.6787]
 [0.5469 0.4854 0.9595 0.7577]
 [0.9649 0.1419 0.0357 0.3922]]


#### For generating data

In [ ]:
L = 3 #{3,5,10,15}
n = 3 #{3,4,5,6}

In [ ]:
def generate_transition_matrix(L):
    matrix = np.random.rand(L+2, L).round(4)  # Generate random values between 0 and 1, rounded to 4 decimals
    return matrix

tmatrix = generate_transition_matrix(L)
print("Transition Matrix:\n", tmatrix)

# bringing row that has P(wi|w0) values at the top to ease indexing
row0 = tmatrix[len(tmatrix) - 2]
tmatrix = np.delete(tmatrix, len(tmatrix) - 2, axis = 0)
tmatrix = np.insert(tmatrix, 0, row0, axis = 0)
print()
print(tmatrix)

In [ ]:
nltk.download('words')

def generate_meaningful_vocabulary(num_words):
    word_list = words.words()  # List of English words from nltk
    vocabulary = set()
    
    while len(vocabulary) < num_words:
        word = random.choice(word_list)
        vocabulary.add(word)
    
    return sorted(vocabulary)

vocab = generate_meaningful_vocabulary(L)
print("Vocabulary:", vocab)

#### Normalising Transition Matrix

In [5]:
for i in range(0, L+1):
    sum = 0
    if(i == 0):
        sum = np.sum(tmatrix[i, :])
        tmatrix[i, :] /= sum
        continue
    sum = np.sum(tmatrix[i, :]) + tmatrix[L+1, i-1]
    tmatrix[i, :] /= sum
    tmatrix[L+1, i-1] = tmatrix[L+1, i-1] / sum
print(tmatrix)

[[0.3033327  0.25353228 0.2077235  0.23541152]
 [0.20900258 0.0520854  0.13940115 0.28062   ]
 [0.03186587 0.31722064 0.29927771 0.30525869]
 [0.10155709 0.34905007 0.2888816  0.24749298]
 [0.17407773 0.15450234 0.3054079  0.24117516]
 [0.31889087 0.0463771  0.01301827 0.12483687]]


### Search Algorithms

Function to calculate cumulated score. for eg: **s(w0, w1, w2) = P(w1|w0).P(w2|w1)**

In [6]:
def calculate_score(sentence):
    score = 1
    L = len(tmatrix)
    for i in range(len(sentence) - 1):
        r = sentence[i]
        c = sentence[i + 1]
        if c == -1:
            score = score * tmatrix[c][r - 1]
            # print(tmatrix[c][r - 1])
            break
        score = score * tmatrix[r][c - 1]
        # print(tmatrix[r][c - 1])
    return score

In [7]:
# example test
print(calculate_score([0, 1, 2]))

0.015799204666223723


***

**IDDFS (Iterative Deepening Depth-First Search)**

In [8]:
def iddfs(maxdepth):
    L = len(vocab)

    # initialize best_sentence and best_score
    best_sentence = []
    best_score = -float('inf')
    
    # iddfs
    for depth in range(1, maxdepth + 1):
        print(f"Running DFS with depth: {depth}")  # Debug output
        print("best sentence upto now", best_sentence)
        best_sentence, best_score = dfs_iterative([0], best_score, L, depth)

    best_sentence = " ".join([("<sos>" if w == 0 else ("<eos>" if w == -1 else vocab[w-1])) for w in best_sentence])
    return best_sentence, best_score

def dfs_iterative(start_sentence, best_score, L, max_depth):
    stack = [(start_sentence, 0)]  # stack of tuples (current_sentence, current_depth)
    local_best_sentence = []
    local_best_score = -float('inf')
    
    while stack:
        current_sentence, depth = stack.pop()
        
        # print(f"Current Sentence: {current_sentence}, Depth: {depth}")
        
        # if we have reached the maximum depth, calculate the score
        if depth == max_depth:
            sentence_with_eos = current_sentence + [-1]
            score = calculate_score(sentence_with_eos)
            print(f"Evaluating Sentence: {sentence_with_eos}, Score: {score}")  # Debug output
            if score > local_best_score:
                local_best_score = score
                local_best_sentence = sentence_with_eos
        else:
            # add each possible word to the stack for further exploration
            for word_index in range(1, L+1):
                new_sentence = current_sentence + [word_index]
                stack.append((new_sentence, depth + 1))

    print("local best sent", local_best_sentence)
    
    return local_best_sentence, local_best_score

In [9]:
sentence, score = iddfs(n)
print("Best Sentence:", sentence)
print("Best Score:", score)

Running DFS with depth: 1
best sentence upto now []
Evaluating Sentence: [0, 4, -1], Score: 0.029388037572980755
Evaluating Sentence: [0, 3, -1], Score: 0.0027042004699956374
Evaluating Sentence: [0, 2, -1], Score: 0.011758090909111805
Evaluating Sentence: [0, 1, -1], Score: 0.0967300290763913
local best sent [0, 1, -1]
Running DFS with depth: 2
best sentence upto now [0, 1, -1]
Evaluating Sentence: [0, 4, 4, -1], Score: 0.0070876646621407255
Evaluating Sentence: [0, 4, 3, -1], Score: 0.0009359684909508888
Evaluating Sentence: [0, 4, 2, -1], Score: 0.001686810586648104
Evaluating Sentence: [0, 4, 1, -1], Score: 0.013068116826336424
Evaluating Sentence: [0, 3, 4, -1], Score: 0.006417877071266831
Evaluating Sentence: [0, 3, 3, -1], Score: 0.0007811937469753655
Evaluating Sentence: [0, 3, 2, -1], Score: 0.0033626131476973955
Evaluating Sentence: [0, 3, 1, -1], Score: 0.006727256007573089
Evaluating Sentence: [0, 2, 4, -1], Score: 0.009661491402355922
Evaluating Sentence: [0, 2, 3, -1], Sc

---

**Greedy Search**

In [10]:
def greedy_search(vocab, tmatrix, n, L):
    sentence = [0]  # start with <SoS>, which is indexed as 0
    current_word = 0  # start with <SoS>
    score = 1;

    for _ in range(n + 1):  # we need n words between <SoS> and <EoS>
        if len(sentence) == n + 1:
            sentence.append(-1)
            score = score * tmatrix[int(L) + 1][sentence[n]-1]
            break
        next_word = np.argmax(tmatrix[current_word]) + 1
        score = score * np.max(tmatrix[current_word])
        # print(score)
        # print(next_word, current_word)
        sentence.append(next_word)
        current_word = next_word

    s = ""
    for i in range(len(sentence)):
        if sentence[i] == 0:
            s = s + "<sos> "
        elif sentence[i] == -1:
            s = s + "<eos>"
        else:
            s = s + vocab[sentence[i]-1] + " "
    return s, score

In [11]:
sentence, score = greedy_search(vocab, tmatrix, n, L)
print("Best Sentence", sentence)
print("Best Score", score)

Best Sentence <sos> green airport at grass <eos>
Best Score 0.00042083261917621675


---

**UCS (Uniform Cost Search)**

In [12]:
def uniform_cost_search(vocab, tmatrix, n):
    # priority queue to store (-cumulative_score, current_word_index, path)
    pq = [(-1, 0, [0])]  # start with <SoS> which is indexed as 0
    best_score = {}

    while pq:
        current_score, idx, path = heapq.heappop(pq)
        current_score = -current_score
        print(path)
        print(current_score)

        # if we have reached the end of the sentence
        if len(path) == n + 2:  # n words + <SoS> + <EoS>
            path[len(path) - 1] = -1
            print(path)
            s = ""
            for i in range(len(path)):
                if path[i] == 0:
                    s = s + "<sos> "
                elif path[i] == -1:
                    s = s + "<eos>"
                else:
                    s = s + vocab[path[i]-1] + " "

            return s, current_score

        # expand to the next word
        if(len(path) != n+1):
            current_word = path[-1]
            for next_word in range(1, len(vocab) + 1):
                new_path = path + [next_word]
                new_score = calculate_score(new_path)
    
                if tuple(new_path) not in best_score or new_score > best_score[tuple(new_path)]:
                    best_score[tuple(new_path)] = new_score
                    heapq.heappush(pq, (-new_score, idx + 1, new_path))
        else:
            current_word = path[-1]
            new_path = path + [-1]
            new_score = calculate_score(new_path)
    
            if tuple(new_path) not in best_score or new_score > best_score[tuple(new_path)]:
                best_score[tuple(new_path)] = new_score
                heapq.heappush(pq, (-new_score, idx + 1, new_path))

    return None, float('-inf')  # If no valid sentence is found

sentence, score = uniform_cost_search(vocab, tmatrix, n)
print(sentence)
print(score)

[0]
1
[0, 1]
0.3033326997402268
[0, 2]
0.25353228156877655
[0, 4]
0.2354115187226763
[0, 3]
0.20772349996832032
[0, 1, 4]
0.08512122260209748
[0, 2, 2]
0.08042567326556672
[0, 2, 4]
0.07739293100148292
[0, 2, 3]
0.075876559869441
[0, 3, 2]
0.072505901677306
[0, 4, 3]
0.07189653761161406
[0, 1, 1]
0.06339731618603986
[0, 3, 3]
0.06000749614371272
[0, 4, 4]
0.056775410680896275
[0, 3, 4]
0.05141010809484703
[0, 1, 3]
0.04228492720947441
[0, 4, 1]
0.04097990246981942
[0, 4, 2]
0.03637163038736578
[0, 2, 3, 2]
0.02648471834118402
[0, 1, 4, 3]
0.025996693855782706
[0, 2, 2, 2]
0.0255126837505504
[0, 4, 3, 2]
0.02509549130359078
[0, 2, 2, 4]
0.02455063530085934
[0, 2, 2, 3]
0.024069611076013804
[0, 2, 4, 3]
0.02363641254604923
[0, 3, 2, 2]
0.023000368718499593
[0, 3, 2, 4]
0.022133056236429652
[0, 2, 3, 3]
0.021919341694406583
[0, 3, 2, 3]
0.021699399995394678
[0, 3, 1]
0.021095793582458965
[0, 3, 3, 2]
0.02094562057716582
[0, 4, 3, 3]
0.02076958651348163
[0, 1, 4, 4]
0.020529124475796308
[0

---

**A\* Search**

h(x) calculates proximity score to goal

In [13]:
def h_heuristic(path, n, vocab_size, tmatrix):
    remaining_words = (n + 2) - len(path)  # remaining words to complete the sentence including <EoS>

    # print(path[-1] - 1)
    # print(vocab_size + 1)
    # print(remaining_words)
    
    if remaining_words == 1:  # only <EoS> is left to predict
        return float(tmatrix[vocab_size + 1][path[-1] - 1])
    
    # if more than one word is left, we optimistically assume the best-case transitions
    max_prob = 1
    row = path[-1]
    last_word = -1
    for _ in range(remaining_words - 1):
        max_prob *= max(tmatrix[row])  # best possible transition at each step
        # print(max_prob)
        row = np.argmax(tmatrix[row])
        last_word = row 
        # print("row", row)
        # print("last word", last_word) 
    
    # final transition to <EoS>
    max_prob *= tmatrix[vocab_size + 1][last_word]
    
    return max_prob

In [14]:
def a_star_search(vocab, tmatrix, n):
    vocab_size = len(vocab)
    # Priority queue to store (-(g+h), g, current_word_index, path)
    pq = [(-(1 + h_heuristic([0], n, vocab_size, tmatrix)), 1, 0, [0])]  # Start with <SoS> which is indexed as 0 in the vocabulary
    best_score = {}

    while pq:
        # Pop the element with the highest priority (lowest g+h value)
        f_score, g_score, idx, path = heapq.heappop(pq)
        g_score = -g_score  # Convert back to positive accumulated score
        print(f"Path: {path}, g: {g_score}, f: {-f_score}")

        # If we've reached the end of the sentence (i.e., we have n words + <EoS>)
        if len(path) == n + 2:
            g_score = calculate_score(path)
            sentence = " ".join([("<sos>" if w == 0 else ("<eos>" if w == -1 else vocab[w-1])) for w in path])
            return sentence, g_score

        # Expand to the next word
        if len(path) != n + 1:
            current_word = path[-1]
            for next_word in range(1, vocab_size + 1):  # Exclude <SoS> which is index 0
                new_path = path + [next_word]
                new_g = calculate_score(new_path)  # Update g (accumulated score)
                h_score = h_heuristic(new_path, n, vocab_size, tmatrix)  # Calculate heuristic h
    
                if tuple(new_path) not in best_score or new_g > best_score[tuple(new_path)]:
                    best_score[tuple(new_path)] = new_g

                    # Push new (f = g + h, g, current index, path) to priority queue
                    heapq.heappush(pq, (-(new_g + h_score), -new_g, idx + 1, new_path))
                    
        # for last word
        else:
            current_word = path[-1]
            new_path = path + [-1]
            new_g = calculate_score(new_path)  # Update g (accumulated score)
            h_score = 0  # Calculate heuristic h

            if tuple(new_path) not in best_score or new_g > best_score[tuple(new_path)]:
                best_score[tuple(new_path)] = new_g
                # Push new (f = g + h, g, current index, path) to priority queue
                heapq.heappush(pq, (-(new_g + h_score), -new_g, idx + 1, new_path))            

    return None, float('-inf')  # If no valid sentence is found

sentence, score = a_star_search(vocab, tmatrix, n)
print(f"Best Sentence: {sentence}")
print(f"Best Score: {score}")

Path: [0], g: -1, f: 1.0026997217767637
Path: [0, 1], g: 0.3033326997402268, f: 0.30676407209698425
Path: [0, 2], g: 0.25353228156877655, f: 0.2549733059804652
Path: [0, 4], g: 0.2354115187226763, f: 0.23880545127363254
Path: [0, 3], g: 0.20772349996832032, f: 0.20930911452377846
Path: [0, 2, 2], g: 0.08042567326556672, f: 0.09153845898243926
Path: [0, 1, 4], g: 0.08512122260209748, f: 0.08961431403539102
Path: [0, 2, 3], g: 0.075876559869441, f: 0.08810438518692612
Path: [0, 4, 3], g: 0.07189653761161406, f: 0.0841243629290992
Path: [0, 3, 2], g: 0.072505901677306, f: 0.08361868739417855
Path: [0, 2, 4], g: 0.07739293100148292, f: 0.08188602243477647
Path: [0, 3, 3], g: 0.06000749614371272, f: 0.07223532146119785
Path: [0, 1, 1], g: 0.06339731618603986, f: 0.06793997267938802
Path: [0, 4, 4], g: 0.056775410680896275, f: 0.06126850211418981
Path: [0, 3, 4], g: 0.05141010809484703, f: 0.05590319952814057
Path: [0, 1, 3], g: 0.04228492720947441, f: 0.054512752526959536
Path: [0, 1, 4, 1]